In [23]:
!python -m pip install pyserial

In [27]:
import serial as pyserial

In [28]:
#0. Setup

import time
import sqlite3
import threading
from datetime import datetime
from matplotlib import pyplot
from matplotlib.dates import DateFormatter, AutoDateFormatter, AutoDateLocator

import matplotlib as plt
import json as json
#!pip install serial
#import pyserial
#!pip install tzlocal as tzlocal  #only works with python versions 3.8 and later

!pip install beebotte
import beebotte as beebotte
from beebotte import *

!pip install paho-mqtt
import paho.mqtt.client as mqtt

DBFILE = 'data.db'
BAUD = 115200
lines = {}
portname = 'COM12'

port = pyserial._init_(port=portname, baudrate=BAUD, bytesize=EIGHTBITS, parity=PARITY_NONE, stopbits=STOPBITS_ONE, timeout=None, xonxoff=False, rtscts=False, write_timeout=None, dsrdtr=False, inter_byte_timeout=None, exclusive=None)
pyserial.open(port)

AttributeError: module 'serial' has no attribute '_init_'

In [ ]:
#1. Send signal to 5 nodes to start CO2 sensor calibration

def start_calibration_msg():
    data = {'type': 'start_calibration'}
    #data = data.encode('utf-8')
    print(data)
    pyserial.write(data)

start_calibration_msg()

In [ ]:
#2. Accept information from nodes in JSON

def record_data(conn, resource, value):
        """ Record the given datapoint at the current time """
        c = conn.cursor()
        c.execute("INSERT INTO data(timestamp, resource, value) VALUES (?, ?, ?)",
                  (time.time(), resource, value))
        conn.commit()

def read_serial(port):
        """ Read from the serial port and process any data received """
        # Open db inside the thread, since a connection object cannot
        # cross threads
        conn = sqlite3.connect(DBFILE)

        while True:
                # Read lines and process any data on lines starting with
                # "DATA:". All lines are printed, so any debug output
                # will also be visible.
                line = port.readline().decode().strip()
                print(line)
                parts = line.split(":")
                if len(parts) == 3 and parts[0] == "DATA":
                        record_data(conn, parts[1], parts[2])
                                            
def main():
        # Open the serial port
        try:
                port = serial.Serial(SERIAL, BAUD)
        except serial.SerialException as e:
                print("Failed to open serial port: " + str(e))
                return
        print("Serial port opened, waiting for data...");

        # Open the database
        conn = sqlite3.connect(DBFILE)

        # Create the table if it does not exist yet
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS data
                     (resource TEXT, timestamp INTEGER, value REAL)''')

        # Draw a line for all resources in the database
        c = conn.cursor()
        c.execute("SELECT DISTINCT resource FROM data")

        # Let matplotlib display times in the system timezone
        try:
                import tzlocal
                timezone = tzlocal.get_localzone()
                pyplot.rcParams['timezone'] = timezone
        except ImportError:
                print("tzlocal module not found, showing times in UTC")


        # Prevent confusing +2e1 offset notation when temperatures are
        # very close together
        pyplot.gca().yaxis.get_major_formatter().set_useOffset(False)

        # Run the serial processing in a separate thread. This allows
        # calling pyplot.show() below, which does not return until the
        # window is closed. By making this a daemon thread, it will be
        # automatically killed when the main thread completes (so when
        # the window is closed).
        t = threading.Thread(target = read_serial, args = (port,))
        t.daemon = True
        t.start()

main()


In [ ]:
#3. Send to beebotte in JSON

def on_connect(client, data, flags, rc):
    #client.subscribe("R1/temperature", 1)
    client.subscribe("R2/temperature", 1)
    #client.subscribe("R3/temperature", 1)
    #client.subscribe("R4/temperature", 1)
    #client.subscribe("R5/temperature", 1)

def on_message(client, data, msg):
    print(msg.topic + " " + str(msg.payload))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.username_pw_set("token:token_B9ukZA1NMfQkrAqI") #node 2
client.connect("mqtt.beebotte.com", 1883, 6)

client.loop_start()

while 1:
    tempdata1 = 1
    tempdata2 = 2
    tempdata3 = 3
    tempdata4 = 4
    tempdata5 = 5
    #client.publish("R1/temperature", tempdata1, 1)
    client.publish("R2/temperature", tempdata2, 1)
    #client.publish("R3/temperature", tempdata3, 1)
    #client.publish("R4/temperature", tempdata4, 1)
    #client.publish("R5/temperature", tempdata5, 1)
    
    time.sleep(1)
    

In [ ]:
#3. Send to beebotte in JSON

mqtt.disconnect()

def on_connect(client, data, flags, rc):
    client.subscribe("temperature", 1)

def on_message(client, data, msg):
    print(msg.topic + " " + str(msg.payload))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
client.username_pw_set("token:token_B9ukZA1NMfQkrAqI") #node 2
client.connect("mqtt.beebotte.com", 1883, 6)

client.loop_start()

while 1:
    tempdata2 = 2
    client.publish("temperature", tempdata2, 2)
    time.sleep(1)
    
    
    
    


from beebotte import *

# Step 3: Authenticate with Beebotte
# Replace with your actual Beebotte API and Secret keys
bbt = BBT('UUtkSBvfRSUs4EhSsHL9JTwq', 'IMYOVMvxv5utLRdj5EXSJo9HJxmCbrAp')

# A function to send data
def send_temperature_data(channel, temperature):
    try:
        # Sending data to Beebotte
        bbt.write(channel, 'temperature', temperature)
        print(f"Data {temperature}°F sent successfully to channel {channel}")
    except Exception as e:
        # Error handling
        print(f"An error occurred while sending data to channel {channel}: ", e)

# Step 4: Sending Data to Multiple Channels
send_temperature_data('R1', 41)
send_temperature_data('R2', 44)
send_temperature_data('R3', 47)
send_temperature_data('R4', 51)
send_temperature_data('R5', 55)

    
